# TP1- Revision sobre los SGBD relacionales

Para revisar algunos conceptos de los SGBD relacionales (en particular el modelo relacional y el lenguaje SQL), vamos a crear una pequeña Base de Datos relacional desde un programa en Python y a partir de datos que se encuentran en varios archivos textos.

La base de datos que queremos construir está representada por el modelo Entidad-Asociación siguiente:

<img src="modeloEA-1.png">

## 1. Transformación del Modelo Entidad-Asociación hacia Modelo Relacional

Proponer un modelo relacional que corresponde al modelo Entidad-Asociación.

Medios Prensa(PK Nombre, Dueño, Numero Followers)
Noticias (PK ID, Titulo, Texto, #publicar, fecha)


...

## 2. Crear la estructura de la base de datos desde un programa Python

In [1]:
import sys
print(sys.version)

3.6.1 |Anaconda 4.4.0 (64-bit)| (default, May 11 2017, 13:09:58) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]


### 2.1 Conectarse a MySQL desde Python

In [2]:
import mysql.connector
config= {
    'user': 'root',
    'password': 'root',
    'host': 'localhost',
}
conector = mysql.connector.connect(**config)
cursor = conector.cursor()
try:
    cursor.execute("CREATE DATABASE tp1")

except Exception as db_creation:
    print(db_creation)

cursor.close()
conector.close()    



# 2.2 Crear las tablas

In [3]:
import mysql.connector
config= {
    'user': 'root',
    'password': 'root',
    'host': 'localhost',
    'database': 'tp1',
}
conector = mysql.connector.connect(**config)
cursor = conector.cursor()

cursor.execute('CREATE TABLE medios_prensa ('
    'nombre VARCHAR(20) NOT NULL PRIMARY KEY,'
    'dueno VARCHAR(150),'
    'numero_followers INTEGER);'
    )
conector.commit()
cursor.execute('CREATE TABLE noticias ('
    'id INTEGER PRIMARY KEY  AUTO_INCREMENT,'
    'titulo VARCHAR(200),'
    'texto TEXT,'
    'publicar VARCHAR(20) NOT NULL,'
    'fecha DATETIME,'
    'FOREIGN KEY (publicar) REFERENCES medios_prensa(nombre)'
    ');'
)  

conector.commit()
cursor.close()
conector.close()

# 3. Insertar los datos en la base de datos

## 3.1 Leer los archivos textos

In [4]:
import mysql.connector
config= {
    'user': 'root',
    'password': 'root',
    'host': 'localhost',
    'database': 'tp1',
}
conector = mysql.connector.connect(**config)
cursor = conector.cursor()


medios_data=[]

with open("medios.csv","r") as file:
    file.readline()
    datos = file.readlines()
    for d in datos:
        lista = d.split(',')
        medios_data.append(tuple((lista[0],lista[1],lista[2].replace('\n',''))))
        
query = 'INSERT INTO medios_prensa(nombre,dueno,numero_followers) VALUES(%s,%s,%s);'
try:
    cursor.executemany(query,medios_data)
    conector.commit()
except Exception as ex:
    print('exception inserting medios_prensa')
    


In [5]:
import mysql.connector
config= {
    'user': 'root',
    'password': 'root',
    'host': 'localhost',
    'database': 'tp1',
}
conector = mysql.connector.connect(**config)
cursor = conector.cursor()

noticias_data=[]
with open("noticias.txt","r") as file:
    lineas=file.readlines()
    i=0
    c = 0
    noticia={}
    while(i<len(lineas)):
        if(lineas[i] != '----------\n'):
            if(c == 0):
                noticia['fecha'] = lineas[i].replace('\n','')
            elif(c == 1):
                noticia['medio'] = lineas[i].replace('\n','')
            elif(c == 2):
                noticia['titulo'] = lineas[i].replace('\n','')
            elif(c == 3):
                noticia['contenido'] = lineas[i].replace('\n','')
            c = c+1
        else:
            #noticias.append(noticia)
            noticias_data.append((noticia['fecha'],noticia['medio'],noticia['titulo'],noticia['contenido']))
            c = 0
        i = i +1;

try:
    query = 'INSERT INTO noticias(fecha,publicar,titulo,texto) VALUES(%s,%s,%s,%s);'
    cursor.executemany(query,noticias_data) 
    conector.commit()
except Exception as ex:
    print('exception inserting noticias',ex)
cursor.close()
conector.close()

## 3.2 Leer los archivos textos y insertar los datos

Está hecho arriba


# 4. Consultar la base de datos

Listar los medios de prensa en Chile ordenado según su número de followers en Twitter.

In [6]:
import mysql.connector
config= {
    'user': 'root',
    'password': 'root',
    'host': 'localhost',
    'database': 'tp1',
}
conector = mysql.connector.connect(**config)
cursor = conector.cursor()

query='SELECT nombre, numero_followers FROM medios_prensa ORDER BY numero_followers DESC'

cursor.execute(query)
for (nombre,numero_followers) in cursor:
    print(nombre,numero_followers)

cursor.close()
conector.close()



24HorasTVN 3286603
TVN 2910624
T13 2869408
CNNChile 2814705
biobio 2756031
Cooperativa 2638045
canal13 2089537
latercera 1849215
Emol 1805524
adnradiochile 1660965
thecliniccl 1637257
elmostrador 1533717
chilevision  1264464
terraChile 1190512
Mega 1016622
ahoranoticiasAN 1012851
ciper 902201
PublimetroChile 876749
lacuarta 763174
deportes13cl 610516
La_Segunda 563406
LaRedTV 531088
el_dinamo 418425
chvdeportes 351803
nacioncl 318346
revistaQP 298495
DiarioLaHora 296059
El_Ciudadano 277703
CadenaChile 277201
ElGraficoChile 268826
difamadores 248538
biobiodeportes 235075
Emol_Deportes 220974
LaSerenaOnline 212759
soychilecl 208424
Dfinanciero 133828
elquintopoder 130323
mercuriovalpo 129761
tvn_deportes 128555
GlamoramaCL 121128
eldia_cl 106078
ElPeriscopioCL 105532
elobservatodo 99838
laterceracom 96775
elsurcl  95662
uchileradio 94999
soyconcepcion 94205
RadioPudahuel 88523
biobiotv 88475
soyvalparaiso 84203
AustralTemuco 83479
hoyxhoycl 83193
RadioDuna 82363
radiozero977 80399
revist

In [7]:
import mysql.connector
config= {
    'user': 'root',
    'password': 'root',
    'host': 'localhost',
    'database': 'tp1',
}
conector = mysql.connector.connect(**config)
cursor = conector.cursor()

query='SELECT COUNT(*) FROM ( SELECT dueno AS total FROM medios_prensa GROUP BY(dueno) ORDER BY dueno) as result;'

cursor.execute(query)
print("total",cursor.fetchall()[0][0])

cursor.close()
conector.close()


total 158


Contar cuantes medios de prensa posee cada dueño.

In [8]:
import mysql.connector
config= {
    'user': 'root',
    'password': 'root',
    'host': 'localhost',
    'database': 'tp1',
}
conector = mysql.connector.connect(**config)
cursor = conector.cursor()

query='SELECT dueno,COUNT(dueno) AS total FROM medios_prensa GROUP BY(dueno) ORDER BY DUENO'

cursor.execute(query)
for (nombre) in cursor:
    print(nombre)

cursor.close()
conector.close()


('"Pedro Cayuqueo Enama"', 1)
('Aconcagua News', 1)
('Agencia Uno', 1)
('Albavisión (MX)', 1)
('Araucanía Noticias', 1)
('AraucaníaTV', 1)
('Benjamín González', 1)
('Betazeta Networks S.A.', 1)
('Bethia', 2)
('BIO-BIO comunicaciones', 1)
('Biobio Comunicaciones', 1)
('BS Comunicaciones', 1)
('Carta Abierta', 1)
('Cauquenes NET', 1)
("Centro Cultural de Comunicación y Educación Radiofónica ''Chasqui''", 1)
('CEPT Comunicaciones', 1)
('Chañarcillo', 1)
('Chillan Activo', 1)
('Chillan Noticias', 1)
('Chiloe Al día', 1)
('Clave9', 1)
('Colectivo de Comunicacion Mapuche Mapuexpress', 1)
('Compañía Chilena de Comunicaciones', 1)
('Compañía de Radio y TV Nuevo Mundo S.A', 1)
('Comunicaciones C Y V S A ', 1)
('Comunidad Sitio Cero', 1)
('Consorcio Periodístico El Epicentro S.A', 1)
('Consorcio Periodístico El Norte SpA', 1)
('CoolTV', 1)
('Diario El Sur S.A', 1)
('Diario El Sur S.A.', 1)
('Diario ELDIA', 1)
('Diario en Acción', 1)
('Diario Naciemiento', 1)
('Dichato al Dia', 1)
('Ediciones Gir

Mostrar los últimos 10 títulos de noticias del medio "La Tercera"

In [9]:
import mysql.connector
config= {
    'user': 'root',
    'password': 'root',
    'host': 'localhost',
    'database': 'tp1',
}
conector = mysql.connector.connect(**config)
cursor = conector.cursor()

query='SELECT * FROM noticias WHERE publicar = \'latercera\' ORDER BY fecha DESC LIMIT 10;'

cursor.execute(query)
for (nombre) in cursor:
    print(nombre)

cursor.close()
conector.close()

(50, 'Cabo de Carabineros dispara a su pareja en hotel de Puerto Montt', 'Personal de la PDI investiga un grave incidente registrado durante la madrugada de este domingo en el\xa0Hotel Vicente Costanera de la ciudad de\xa0Puerto Montt, en la región de Los Lagos. Los primeros antecedentes indican que estando en una habitación del mencionado recinto, un Cabo Primero de la\xa0Quinta Comisaría de la ciudad habría disparado a su pareja tras una discusión. Tras ello, el funcionario se habría autoinferido un disparo en la cabeza.\xa0 Debido al complejo incidente, ambos fueron trasladados hasta el Hospital Regional de Puerto Montt, donde permanecen con diagnóstico reservado.\xa0 Desde la institución informaron que se inició un sumario interno.', 'latercera', datetime.datetime(2017, 5, 7, 12, 56, 47))


Mostrar los nombre de medios y los títulos en los cuales aparacen la palabra "Valdivia".

In [10]:
import mysql.connector
config= {
    'user': 'root',
    'password': 'root',
    'host': 'localhost',
    'database': 'tp1',
}
conector = mysql.connector.connect(**config)
cursor = conector.cursor()

query='SELECT publicar, titulo FROM noticias WHERE titulo LIKE "%valdivia%" ;'

cursor.execute(query)
for (nombre) in cursor:
    print(nombre)

cursor.close()
conector.close()

Por cada medio, mostrar el número de títulos en los cuales aparecen la palabra "Puerto Montt".

In [11]:
import mysql.connector
config= {
    'user': 'root',
    'password': 'root',
    'host': 'localhost',
    'database': 'tp1',
}
conector = mysql.connector.connect(**config)
cursor = conector.cursor()

query='SELECT publicar as medio, COUNT(*) as titulos FROM (SELECT * FROM noticias WHERE titulo LIKE "%Puerto montt%") as result GROUP BY PUBLICAR;'

cursor.execute(query)
for (nombre) in cursor:
    print(nombre)

cursor.close()
conector.close()

('24HorasTVN', 2)
('adnradiochile', 8)
('biobio', 35)
('CNNChile', 1)
('latercera', 1)
('TerraChile', 1)
('thecliniccl', 1)
('TVN', 1)
